In [2]:
import pandas as pd
import numpy as np
import sqlite3 as db
from datetime import datetime
import urllib
import os


con = db.connect(database="../db.sqlite")

In [26]:
cursor = con.cursor()
cursor.execute("select CountryCode, Name from Country")

list = []

for row in cursor.fetchall():
    list.append({
        "CountryCode": row[0],
        "Country": row[1]
    });
    

def getCountryCode(name):
    
    if name == "United Kingdom":
        return "GBR"
    elif name == "Russian Federation":
        return "RUS"
    elif name == "Korea":
        return "KOR"
    elif name == "Slovak Republic":
        return "SVK"
    
    for c in list:
        if c["Country"] == name:
            return c["CountryCode"]
        
    return None

In [16]:
xl = pd.ExcelFile("oecd 2021.xlsx")

dfMonthly = xl.parse("Monthly")
dfAnnually = xl.parse("Annually")

dfMonthly["Type"] = "Monthly"
dfAnnually["Type"] = "Annually"


df = pd.concat([dfMonthly, dfAnnually])
df["Source"] = "oecd"
df["Profession"] = "General"

In [31]:
dfKeys = df.groupby("Country").groups.keys()


for key in dfKeys:
    if getCountryCode(key) == None:
        print(key + " does not exist in db")

In [48]:

for year in range(2010,2021):
    yearStr = str(year)
    yearDf = df[["Country", "Profession", "Type", yearStr, "Source"]]
    yearDf["Date"] = datetime(year, 1, 1)
    yearDf = yearDf.rename(columns={yearStr:"GrossLocalPrice"})
    yearDf["Country"] = yearDf["Country"].apply(getCountryCode)
    yearDf = yearDf.rename(columns={"Country":"Country_id"})
    
    yearDf = yearDf.loc[yearDf["GrossLocalPrice"] != ".."]
    
    yearDf.to_sql("MinWage", con,if_exists="append", index=False)
    

C:\Users\egonu\AppData\Local\Temp/ipykernel_11516/1233021587.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yearDf["Date"] = datetime(year, 1, 1)


IntegrityError: UNIQUE constraint failed: MinWage.Country_id, MinWage.Date, MinWage.Type